### Red Agents

Scenario 1b has two rules-based Red Agents. The first is our good friend the B_lineAgent. This represents an actor who has inside information, so is able to beeline straight towards the OpServer.

In [1]:
import inspect
from pprint import pprint
from CybORG import CybORG
from CybORG.Agents import *
from CybORG.Shared.Actions import *

path = str(inspect.getfile(CybORG))
path = path[:-10] + '/Shared/Scenarios/Scenario1b.yaml'

env = CybORG(path,'sim')

agent = B_lineAgent()

results = env.reset('Red')
obs = results.observation
action_space = results.action_space

for i in range(16):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    
    print(action)

DiscoverRemoteSystems 10.0.114.144/28
DiscoverNetworkServices 10.0.114.149
ExploitRemoteService 10.0.114.149
PrivilegeEscalate User1
DiscoverNetworkServices 10.0.214.51
ExploitRemoteService 10.0.214.51
PrivilegeEscalate Enterprise1
DiscoverRemoteSystems 10.0.214.48/28
DiscoverNetworkServices 10.0.214.56
ExploitRemoteService 10.0.214.56
PrivilegeEscalate Enterprise2
DiscoverNetworkServices 10.0.62.185
ExploitRemoteService 10.0.62.185
PrivilegeEscalate Op_Server0
Impact Op_Server0
Impact Op_Server0


This agent runs along a predetermined path to the Op_Server, but is smart enough able to recover its position if interrupted. We can see below after Blue Team restores some hosts, the agent works out where the error in and re-exploits its way to the Op_Server.

In [2]:
action = Restore(hostname='Op_Server0',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise2',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise1',session=0,agent='Blue')
env.step(action=action,agent='Blue')

for i in range(12):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
            
    print(action)
    print('Success:',obs['success'])

Impact Op_Server0
Success: FALSE
PrivilegeEscalate Op_Server0
Success: FALSE
ExploitRemoteService 10.0.62.185
Success: FALSE
ExploitRemoteService 10.0.214.56
Success: TRUE
PrivilegeEscalate Enterprise2
Success: TRUE
DiscoverNetworkServices 10.0.62.185
Success: TRUE
ExploitRemoteService 10.0.62.185
Success: TRUE
PrivilegeEscalate Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE


The other red agent is the MeanderAgent. This performs a breadth first search on all known hosts, scanning each one in turn, before attempting a mix of exploit and privilege escalate on the rest. This is an extremely slow agent in contrast to the laser-focussed B_lineAgent.

In [3]:
agent = RedMeanderAgent()

results = env.reset('Red')
obs = results.observation
action_space = results.action_space

for i in range(46):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    
    print(action)

DiscoverRemoteSystems 10.0.153.64/28
DiscoverNetworkServices 10.0.153.66
DiscoverNetworkServices 10.0.153.72
DiscoverNetworkServices 10.0.153.68
DiscoverNetworkServices 10.0.153.74
DiscoverNetworkServices 10.0.153.78
PrivilegeEscalate User0
ExploitRemoteService 10.0.153.66
ExploitRemoteService 10.0.153.72
PrivilegeEscalate User1
DiscoverNetworkServices 10.0.172.38
ExploitRemoteService 10.0.172.38
PrivilegeEscalate Enterprise1
DiscoverRemoteSystems 10.0.172.32/28
DiscoverNetworkServices 10.0.172.34
DiscoverNetworkServices 10.0.172.46
DiscoverNetworkServices 10.0.172.36
ExploitRemoteService 10.0.172.34
PrivilegeEscalate Enterprise1
PrivilegeEscalate User0
PrivilegeEscalate User1
ExploitRemoteService 10.0.172.46
PrivilegeEscalate Enterprise0
ExploitRemoteService 10.0.172.36
PrivilegeEscalate Enterprise2
DiscoverNetworkServices 10.0.91.114
ExploitRemoteService 10.0.91.114
PrivilegeEscalate Op_Server0
DiscoverRemoteSystems 10.0.91.112/28
DiscoverNetworkServices 10.0.91.113
DiscoverNetworkSe

The Meander Agent is also able to recover from Blue's disruption.

In [4]:
action = Restore(hostname='Op_Server0',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise2',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise1',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise0',session=0,agent='Blue')
env.step(action=action,agent='Blue')

for i in range(24):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    print(env.get_last_action('Red'))

Impact Op_Server0
ExploitRemoteService 10.0.91.114
PrivilegeEscalate Enterprise0
PrivilegeEscalate Enterprise1
PrivilegeEscalate Enterprise2
PrivilegeEscalate Op_Host0
PrivilegeEscalate Op_Host1
PrivilegeEscalate Op_Host2
PrivilegeEscalate Op_Server0
PrivilegeEscalate User0
PrivilegeEscalate User1
PrivilegeEscalate User2
PrivilegeEscalate User3
PrivilegeEscalate User4
ExploitRemoteService 10.0.172.46
PrivilegeEscalate Enterprise0
ExploitRemoteService 10.0.172.38
PrivilegeEscalate Enterprise1
ExploitRemoteService 10.0.172.36
PrivilegeEscalate Enterprise2
ExploitRemoteService 10.0.91.114
PrivilegeEscalate Op_Server0
Impact Op_Server0
Impact Op_Server0


### Blue Agents

The BlueReactRemoveAgent will wait until it sees suspicious activity, before using remove on all the hosts it has flagged. However, due to the 5% change that Red's exploit is missed, Red will always eventually get to the Op_Server.

In [5]:
env = CybORG(path,'sim',agents={'Red':B_lineAgent})

agent = BlueReactRemoveAgent()

results = env.reset('Blue')
obs = results.observation
action_space = results.action_space

for i in range(12):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    print(env.get_last_action('Blue'))

Monitor
Monitor
Monitor
Remove User1
Monitor
Monitor
Remove Enterprise1
Monitor
Remove Enterprise1
Monitor
Remove Enterprise1
Monitor


The BlueReactRestoreAgent is the same as the React agent above, but uses the Restore action.

In [6]:
agent = BlueReactRestoreAgent()

results = env.reset('Blue')
obs = results.observation
action_space = results.action_space

for i in range(12):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    print(env.get_last_action('Blue'))

Monitor
Monitor
Monitor
Restore User1
Monitor
Restore User1
Monitor
Restore User1
Monitor
Monitor
Monitor
Monitor


### Green Agent

An important part of CybORG Scenario1b is the Green agent, which represents the users on the network. The Green Agent is very simple, it only performs a scanning action on random hosts some of the time. This is only visible by Blue Agent.

In [7]:
agent = GreenAgent()

results = env.reset('Green')
obs = results.observation
action_space = results.action_space

for i in range(12):
    print(agent.get_action(obs,action_space))

GreenPortScan
GreenPortScan
Sleep
GreenPortScan
GreenPortScan
GreenPortScan
GreenPortScan
GreenPortScan
Sleep
GreenPortScan
Sleep
Sleep


### Keyboard Agent

The KeyboardAgent allows a human user to manually choose actions. This is useful for getting an intuition for the scenario.

In [8]:
from CybORG.Agents.Wrappers import RedTableWrapper

cyborg = CybORG(path, 'sim',agents={'Blue':BlueMonitorAgent})
env = RedTableWrapper(env=cyborg, output_mode='table')

agent = KeyboardAgent()

results = env.reset('Red')
obs = results.observation
action_space = results.action_space

for i in range(3):
    print(obs)
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation

+----------------+-------------+----------+---------+------------+
|     Subnet     |  IP Address | Hostname | Scanned |   Access   |
+----------------+-------------+----------+---------+------------+
| 10.0.50.144/28 | 10.0.50.150 |  User0   |  False  | Privileged |
+----------------+-------------+----------+---------+------------+

************************************ Turn 1: Observation *************************************

+----------------+-------------+----------+---------+------------+
|     Subnet     |  IP Address | Hostname | Scanned |   Access   |
+----------------+-------------+----------+---------+------------+
| 10.0.50.144/28 | 10.0.50.150 |  User0   |  False  | Privileged |
+----------------+-------------+----------+---------+------------+

********************************* Turn 1: Command Selection **********************************

0 Sleep
1 DiscoverRemoteSystems
2 DiscoverNetworkServices
3 ExploitRemoteService
4 PrivilegeEscalate
5 Impact


----------------------------------------------------------------------------------------------
CHOOSE A COMMAND:  1


You chose DiscoverRemoteSystems.


******************************** Turn 1: Parameter Selection *********************************


------------------------------------- Session Selection --------------------------------------
Automatically choosing 0 as it is the only option.
-------------------------------------- Agent Selection ---------------------------------------
Automatically choosing Red as it is the only option.
-------------------------------------- Subnet Selection --------------------------------------
Automatically choosing 10.0.50.144/28 as it is the only option.
+----------------+-------------+-----------------+---------+------------+
|     Subnet     |  IP Address |     Hostname    | Scanned |   Access   |
+----------------+-------------+-----------------+---------+------------+
| 10.0.50.144/28 | 10.0.50.150 |      User0      |  False  | Privileged |
| 10.0.50.144/28 | 10.0.50.152 | UNKNOWN_HOST: 3 |  False  |    None    |
| 10.0.50.144/28 | 10.0.50.153 | UNKNOWN_HOST

----------------------------------------------------------------------------------------------
CHOOSE A COMMAND:  2


You chose DiscoverNetworkServices.


******************************** Turn 2: Parameter Selection *********************************


------------------------------------- Session Selection --------------------------------------
Automatically choosing 0 as it is the only option.
-------------------------------------- Agent Selection ---------------------------------------
Automatically choosing Red as it is the only option.
------------------------------------ Ip_address Selection ------------------------------------
0 10.0.50.150
1 10.0.50.153
2 10.0.50.155
3 10.0.50.157
4 10.0.50.152


...

You chose 10.0.50.153.
+----------------+-------------+-----------------+---------+------------+
|     Subnet     |  IP Address |     Hostname    | Scanned |   Access   |
+----------------+-------------+-----------------+---------+------------+
| 10.0.50.144/28 | 10.0.50.150 |      User0      |  False  | Privileged |
| 10.0.50.144/28 | 10.0.50.152 | UNKNOWN_HOST: 3 |  False  |    None    |
| 10.0.50.144/28 | 10.0.50.153 | UNKNOWN_HOST: 0 |   True  |    None    |
| 10.0.50.144/28 | 10.0.50.155 | UNKNOWN_HOST: 1 |  False  |    None    |
| 10.0.50.144/28 | 10.0.50.157 | UNKNOWN_HOST: 2 |  False  |    None    |
+----------------+-------------+-----------------+---------+------------+

************************************ Turn 3: Observation *************************************

+----------------+-------------+-----------------+---------+------------+
|     Subnet     |  IP Address |     Hostname    | Scanned |   Access   |
+----------------+-------------+-----------------+---------+------

----------------------------------------------------------------------------------------------
CHOOSE A COMMAND:  3


You chose ExploitRemoteService.


******************************** Turn 3: Parameter Selection *********************************


------------------------------------- Session Selection --------------------------------------
Automatically choosing 0 as it is the only option.
-------------------------------------- Agent Selection ---------------------------------------
Automatically choosing Red as it is the only option.
------------------------------------ Ip_address Selection ------------------------------------
0 10.0.50.150
1 10.0.50.153
2 10.0.50.155
3 10.0.50.157
4 10.0.50.152


----------------------------------------------------------------------------------------------
CHOOSE A PARAMETER:  1


You chose 10.0.50.153.
